In [3]:
from pyspark.sql import SparkSession,Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Read from GCS") \
    .getOrCreate()

# Path to the GCS bucket/file
gcs_bucket = "gs://sovik-big-query-bucket-testing-1/yesterday_records.csv"

# Read the CSV file from GCS into a DataFrame
df = spark.read.format("csv").option("header", "true").load(gcs_bucket)

# Show the DataFrame contents


+---+-------+------------+
| id|   name|created_date|
+---+-------+------------+
|  1| Name_1|  2024-08-02|
|  2| Name_2|  2024-08-02|
|  3| Name_3|  2024-08-02|
|  4| Name_4|  2024-08-02|
|  5| Name_5|  2024-08-02|
|  6| Name_6|  2024-08-02|
|  7| Name_7|  2024-08-02|
|  8| Name_8|  2024-08-02|
|  9| Name_9|  2024-08-02|
| 10|Name_10|  2024-08-02|
| 11|Name_11|  2024-08-02|
| 12|Name_12|  2024-08-02|
| 13|Name_13|  2024-08-02|
| 14|Name_14|  2024-08-02|
| 15|Name_15|  2024-08-02|
| 16|Name_16|  2024-08-02|
| 17|Name_17|  2024-08-02|
| 18|Name_18|  2024-08-02|
| 19|Name_19|  2024-08-02|
| 20|Name_20|  2024-08-02|
+---+-------+------------+
only showing top 20 rows



In [34]:
spark.stop()

In [3]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime, timedelta

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Read from GCS") \
    .getOrCreate()

# Source path in GCS
source_path = "gs://sovik-big-query-bucket-testing-1/abc"

# Calculate the date today
today_date = datetime.now()
year1, month1, day1 = today_date.year, today_date.month, today_date.day

# Construct the resultant path for today
resultant_path = f"{source_path}/{year1}/{month1:02d}/{day1:02d}/*.csv"

# Initialize empty DataFrame for today
df_v1 = None

# Try to read today's data from GCS
try:
    df_v1 = spark.read.format("csv").option("header", "true").load(resultant_path)
    #df_v1.show()
except Exception as e:
    print(f"No files found for today's date: {today_date.strftime('%Y-%m-%d')}. Error: {e}")

# Calculate the date for yesterday
yesterday_date = today_date - timedelta(days=1)
year0, month0, day0 = yesterday_date.year, yesterday_date.month, yesterday_date.day

# Construct the resultant path for yesterday
resultant_path1 = f"{source_path}/{year0}/{month0:02d}/{day0:02d}/*.csv"

# Initialize empty DataFrame for yesterday
df_v2 = None

# Try to read yesterday's data from GCS
try:
    df_v2 = spark.read.format("csv").option("header", "true").load(resultant_path1)
    #df_v2.show()
except Exception as e:
    print(f"No files found for yesterday's date: {yesterday_date.strftime('%Y-%m-%d')}. Error: {e}")

# Proceed with your logic only if both DataFrames are not None
df_v3=df_v2.withColumn("created_date",lit(today_date.strftime("%Y-%m-%d")))
if df_v1 is not None and df_v2 is not None:
    # Your logic here
    df_v3 = df_v1.union(df_v3).withColumn("created_date", lit(today_date.strftime('%Y-%m-%d')))

else:
    print("One or both DataFrames are empty. Skipping further processing.")
    


# Stop the Spark session to ensure proper shutdown
#spark.stop()

24/08/03 17:26:36 INFO SparkEnv: Registering MapOutputTracker
24/08/03 17:26:36 INFO SparkEnv: Registering BlockManagerMaster
24/08/03 17:26:36 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/08/03 17:26:36 INFO SparkEnv: Registering OutputCommitCoordinator


No files found for today's date: 2024-08-03. Error: [PATH_NOT_FOUND] Path does not exist: gs://sovik-big-query-bucket-testing-1/abc/2024/08/03/*.csv.


One or both DataFrames are empty. Skipping further processing.


+---+-------+------------+
| id|   name|created_date|
+---+-------+------------+
|  1| Name_1|  2024-08-03|
|  2| Name_2|  2024-08-03|
|  3| Name_3|  2024-08-03|
|  4| Name_4|  2024-08-03|
|  5| Name_5|  2024-08-03|
|  6| Name_6|  2024-08-03|
|  7| Name_7|  2024-08-03|
|  8| Name_8|  2024-08-03|
|  9| Name_9|  2024-08-03|
| 10|Name_10|  2024-08-03|
| 11|Name_11|  2024-08-03|
| 12|Name_12|  2024-08-03|
| 13|Name_13|  2024-08-03|
| 14|Name_14|  2024-08-03|
| 15|Name_15|  2024-08-03|
| 16|Name_16|  2024-08-03|
| 17|Name_17|  2024-08-03|
| 18|Name_18|  2024-08-03|
| 19|Name_19|  2024-08-03|
| 20|Name_20|  2024-08-03|
+---+-------+------------+
only showing top 20 rows



In [4]:
df_v3.count()

40

In [6]:
output_path=f"{source_path}/{year1}/{month1:02d}/{day1:02d}/"
df_v3.write.mode("overwrite").csv(path=output_path)

In [16]:
#yesterday_count
#today_count

15

In [19]:
today_date

datetime.datetime(2024, 8, 3, 16, 46, 59, 679789)

In [26]:
df_v3=df_v2.withColumn("created_date",lit(today_date.strftime("%Y-%m-%d")))
if yesterday_count >0  and today_count>0:   
   df_v3=df_v3.union(df_v1)   
df_v3.show() 

+---+-------+------------+
| id|   name|created_date|
+---+-------+------------+
|  1| Name_1|  2024-08-03|
|  2| Name_2|  2024-08-03|
|  3| Name_3|  2024-08-03|
|  4| Name_4|  2024-08-03|
|  5| Name_5|  2024-08-03|
|  6| Name_6|  2024-08-03|
|  7| Name_7|  2024-08-03|
|  8| Name_8|  2024-08-03|
|  9| Name_9|  2024-08-03|
| 10|Name_10|  2024-08-03|
| 11|Name_11|  2024-08-03|
| 12|Name_12|  2024-08-03|
| 13|Name_13|  2024-08-03|
| 14|Name_14|  2024-08-03|
| 15|Name_15|  2024-08-03|
| 16|Name_16|  2024-08-03|
| 17|Name_17|  2024-08-03|
| 18|Name_18|  2024-08-03|
| 19|Name_19|  2024-08-03|
| 20|Name_20|  2024-08-03|
+---+-------+------------+
only showing top 20 rows

